In [1]:
# import packet
import pandas as pd
import numpy as np

In [2]:
# load data
px_long = pd.read_csv('data/stock_data.csv')
financials = pd.read_csv('data/financial_data.csv')

# preprocess data   
# 統一 date 型態
px_long['date'] = pd.to_datetime(px_long['date'], errors='coerce')
financials['date'] = pd.to_datetime(financials['date'], errors='coerce')

# 統一 ticker 欄位名稱 & 型態
financials = financials.rename(columns={"stock_id": "ticker"})
financials = financials.rename(columns={'date': 'report_date'})

for df in [px_long, financials]:
    df['ticker'] = df['ticker'].astype(str).str.strip()

px_long     = px_long.dropna(subset=['date'])
financials = financials.dropna(subset=['report_date'])

# 產生實際可用日（60 天）
financials['effective_date'] = financials['report_date'] + pd.Timedelta(days=60)

px_long = px_long.sort_values('date').reset_index(drop=True)
financials = financials.sort_values('effective_date').reset_index(drop=True)

# financial data 格式修正
# 先確保是字串、去掉空白
t = financials['ticker'].astype(str).str.strip()

# 只有在「沒有 .XXX 後綴」時才補 .TW（你也可以改成只檢查 .TW）
financials['ticker'] = t.where(t.str.contains(r'\.[A-Za-z]+$'), t + '.TW')

In [6]:
# 再確認 ticker 已是字串
px_long['ticker'] = px_long['ticker'].astype(str)
financials['ticker'] = financials['ticker'].astype(str)

# final merge_asof（財報資料平移）
data_merged = pd.merge_asof(
    left=px_long,
    right=financials,
    by='ticker',
    left_on='date',
    right_on='effective_date',
    direction='backward'
)

# 財報補值
cols_to_ffill = data_merged.columns.difference(['ticker', 'date'])

data_merged[cols_to_ffill] = (
    data_merged
    .groupby('ticker')[cols_to_ffill]
    .ffill()
)

In [11]:
data_merged.to_csv('data/merged_data_log60.csv')